In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as ply

## load data and preprocess

save what data in what type file

In [2]:
%%time
questions = pd.read_csv('./question_train_word.csv')
questions_topics = questions.topics.apply(lambda s: s.split(','))
questions_titles = questions.title.astype('U').apply(lambda s: s.split(','))

CPU times: user 28.1 s, sys: 5.83 s, total: 33.9 s
Wall time: 37.9 s


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from sklearn.externals import joblib
from tqdm import tqdm_notebook
from collections import Counter
import os.path

topic_count = Counter([t for ts in questions_topics for t in ts])
topic_most_common = np.array(topic_count.most_common())

tag_index_dict = {}

def tag_index(i):
    tag = topic_most_common[i][0]
    if tag in tag_index_dict:
        return tag_index_list[tag]
    y_index = []
    for topics in questions_topics:
        if tag in topics:
            y_index.append(1)
        else:
            y_index.append(0)
    tag_index_dict[tag] = y_index
    return y_index

def fit_cv(args):
    i, clf = args
    filename = 'sgd_classifier_v4/sgd_{}_{}_{}.pkl'.format(tfidf_ngram_range, sgd_alpha, i)
    if os.path.isfile(filename):
        return i
    
    y = tag_index(i)
    
    length = int(len(y)/10)
    clf.fit(X_train[:-length], y[:-length])
    joblib.dump(clf, filename) 
    return i

In [5]:
tfidf_ngram_range_list = [(1,1), (1,2)]
sgd_alpha_list = [1e-4, 1e-5, 1e-6]

In [6]:
from sklearn.linear_model import SGDClassifier
import multiprocessing

In [8]:
import gc

for _n, tfidf_ngram_range in enumerate(tfidf_ngram_range_list):
    for _m, sgd_alpha in enumerate(sgd_alpha_list):

        
        print('current: {}, {}'.format(tfidf_ngram_range, sgd_alpha))
        vect = TfidfVectorizer(max_df=0.45, min_df=21, ngram_range=tfidf_ngram_range)
        %time X_train = vect.fit_transform(questions.title.astype('U'))
        sgd_train_list = [(i, SGDClassifier(loss="modified_huber", penalty="l2", n_jobs=-1, alpha=sgd_alpha, n_iter=1)) 
                          for i in range(1999)]

        pool = multiprocessing.Pool(processes=4)
        %time l = pool.map(fit_cv, sgd_train_list)
        del pool
        del sgd_train_list
        del X_train
        gc.collect()

current: (1, 2), 1e-05
CPU times: user 2min 53s, sys: 12 s, total: 3min 5s
Wall time: 3min 9s
CPU times: user 408 ms, sys: 176 ms, total: 584 ms
Wall time: 3min 10s
current: (1, 2), 1e-06
CPU times: user 2min 53s, sys: 11.3 s, total: 3min 4s
Wall time: 3min 7s
CPU times: user 10.4 s, sys: 3.81 s, total: 14.2 s
Wall time: 59min 3s


## cross validation

In [7]:
topic_map = {i: v[0] for i, v in enumerate(topic_most_common)}

In [8]:
from tqdm import tqdm_notebook

def predict(clf_list, word_tfidf):
    predict_list = []
    for clf in tqdm_notebook(clf_list):
        result = [1-j[0] for j in clf.predict_proba(word_tfidf)]
        predict_list.append(result)
    return predict_list

def transform(predict_list):
    return np.array(predict_list).T

def top_five(predict_list):
    return np.array(predict_list).argsort()[-5:][::-1]

def transform_predict(predict_list):
    predict_list_t = transform(predict_list)
    result = []
    for p in tqdm_notebook(predict_list_t):
        result.append(list(map(lambda x: topic_map[x], top_five(p))))
    return result

In [9]:
filename = 'sgd_classifier_v4/sgd_{}_{}_{}.pkl'
sgd_list = []

cur_n_gram = (1,1)
cur_alpha = 1e-4

for i in tqdm_notebook(range(1999)):
    sgd_list.append(joblib.load(filename.format(cur_n_gram, cur_alpha, i)))

In [10]:
vect = TfidfVectorizer(max_df=0.45, min_df=21, ngram_range=cur_n_gram)
%time X_train = vect.fit_transform(questions.title.astype('U'))

CPU times: user 47.9 s, sys: 2.17 s, total: 50.1 s
Wall time: 50.5 s


In [11]:
stop_length = int(len(questions_topics)/10)
X_train[-stop_length:].shape

(299996, 57933)

In [12]:
stop_length = int(len(questions_topics)/10)
predict_list = predict(sgd_list, X_train[-stop_length:])

result = transform_predict(predict_list)

In [13]:
from evaluate import evaluate
evaluate(zip(result, questions_topics[-stop_length:]))

(0.35395857880571613, 1.2066986535116266, 0.500881045834571, 347358, 693494)

In [9]:
%pdb

Automatic pdb calling has been turned ON


In [22]:
def predict_cv(tfidf_ngram_range, sgd_alpha):
    print('current: {}, {}'.format(tfidf_ngram_range, sgd_alpha))

    vect = TfidfVectorizer(max_df=0.45, min_df=21, ngram_range=tfidf_ngram_range)
    X_train = vect.fit_transform(questions.title.astype('U'))

    filename = 'sgd_classifier_v4/sgd_{}_{}_{}.pkl'
    sgd_list = []

    for i in tqdm_notebook(range(1999)):
        sgd_list.append(joblib.load(filename.format(tfidf_ngram_range, sgd_alpha, i)))
    
    print('predict')
    stop_length = int(len(questions_topics)/10)
    predict_list = predict(sgd_list, X_train[-stop_length:])
    print('transform')
    result = transform_predict(predict_list)
    return result

In [9]:
FILENAME = 'sgd_classifier_v4/sgd_{}_{}_{}.pkl'

tfidf_ngram_range = (1, 2)
vect_2 = TfidfVectorizer(max_df=0.45, min_df=21, ngram_range=tfidf_ngram_range)
%time X_train_2 = vect_2.fit_transform(questions.title.astype('U'))

CPU times: user 3min, sys: 10.9 s, total: 3min 11s
Wall time: 3min 21s


In [13]:
X_train_2.shape

(2999967, 222485)

In [17]:
result_list_2 = []
for sgd_alpha in [1e-4, 1e-5, 1e-6]:
    sgd_list = []
    for i in tqdm_notebook(range(1999)):
        sgd_list.append(joblib.load(FILENAME.format((1,2), sgd_alpha, i)))
    print('predict')
    stop_length = int(len(questions_topics)/10)

    predict_list = predict(sgd_list, X_train_2[-stop_length:])
    print('transform')
    result_list_2.append(transform_predict(predict_list))

1621/|/ 81%|| 1621/1999 [01:09<00:16, 23.35it/s]
predict
transform
166900/|/ 56%|| 166900/299996 [02:44<02:10, 1016.03it/s]

predict

transform
3326/|/  1%|| 3326/299996 [00:55<1:22:28, 59.95it/s]

predict

transform
6007/|/  2%|| 6007/299996 [00:40<33:01, 148.34it/s]


In [18]:
for i, result in enumerate(result_list_2):
    pd.Series(result).to_csv('sgd_result_v4_cv_2_1e-{}.csv'.format(i+4), header=False, index=False, sep=' ')

In [19]:
from evaluate import evaluate
for result in result_list_2:
    print(evaluate(zip(result, questions_topics[-stop_length:])))

(0.34803699680876915, 1.1847725766168238, 0.49280166807499415, 341755, 693494)
(0.37216923069426944, 1.27924717774402, 0.5248682757168771, 363993, 693494)
(0.3069152746755406, 1.0716297479272252, 0.4300945646249283, 298268, 693494)


5e-05

(0.36020836152515456, 1.2291048630864385, 0.509535771037673, 353360, 693494)

2.5e-05

(0.36882724548243073, 1.2612919275790064, 0.5212518060718622, 361485, 693494)

7.5e-05

(0.35358860114414276, 1.2045755291410178, 0.500506132713477, 347098, 693494)

In [20]:
tfidf_ngram_range = (1, 1)
vect_1 = TfidfVectorizer(max_df=0.45, min_df=21, ngram_range=tfidf_ngram_range)
X_train_1 = vect_1.fit_transform(questions.title.astype('U'))

In [21]:
X_train_1.shape

(2999967, 57933)

In [22]:
result_list_1 = []
for sgd_alpha in [1e-4, 1e-5, 1e-6]:
    sgd_list = []
    for i in tqdm_notebook(range(1999)):
        sgd_list.append(joblib.load(FILENAME.format((1,1), sgd_alpha, i)))

    stop_length = int(len(questions_topics)/10)
    predict_list = predict(sgd_list, X_train_1[-stop_length:])
    
    result_list_1.append(transform_predict(predict_list))



2632/|/  1%|| 2632/299996 [01:06<2:05:09, 39.60it/s]


5705/|/  2%|| 5705/299996 [00:50<43:33, 112.60it/s]


218507/|/ 73%|| 218507/299996 [02:49<01:03, 1287.03it/s]


In [24]:
for i, result in enumerate(result_list_1):
    pd.Series(result).to_csv('sgd_result_v4_cv_1_1e-{}.csv'.format(i+4), header=False, index=False, sep=' ')

In [25]:
for result in result_list_1:
    print(evaluate(zip(result, questions_topics[-stop_length:])))

(0.35395857880571613, 1.2066986535116266, 0.500881045834571, 347358, 693494)
(0.36846068438827806, 1.268284484811129, 0.5193383071807398, 360158, 693494)
(0.29468081308382876, 1.0286364843942957, 0.41299420038241136, 286409, 693494)


## predict: (1,2); 1e-5

In [10]:
print(X_train_2.shape)

def fit(args):
    i, clf = args
    filename = 'sgd_classifier_v4_final/sgd_(1,2)_1e-5_{}.pkl'.format(i)
    if os.path.isfile(filename):
        return i
    
    y = tag_index(i)
    
    clf.fit(X_train_2, y)
    joblib.dump(clf, filename)
    return i

sgd_train_list = [(i, SGDClassifier(loss="modified_huber", penalty="l2", n_jobs=-1, alpha=1e-5, n_iter=1))
                  for i in range(1999)]

pool = multiprocessing.Pool(processes=4)
l = pool.map(fit, sgd_train_list)

(2999967, 222485)


In [12]:
FILENAME_F = 'sgd_classifier_v4_final/sgd_(1,2)_1e-5_{}.pkl'
sgd_list = []
for i in tqdm_notebook(range(1999)):
    sgd_list.append(joblib.load(FILENAME_F.format(i)))

In [15]:
test_questions = pd.read_csv('./question_eval_set.txt', sep='\t', names=['id', 'c', 'w', 'd_c', 'd_w'])
X_test = vect_2.transform(test_questions.w.astype('U'))
X_test.shape

(217360, 222485)

In [17]:
predict_test = predict(sgd_list, X_test)
result_test = transform_predict(predict_test)

160960/|/ 74%|| 160960/217360 [02:13<00:46, 1210.09it/s]


In [18]:
print(result_test[0])
pd.Series(result_test).to_csv('sgd_result_v4_final_2_1e-5.csv', header=False, index=False, sep=' ')

['4610596224687453206', '-7506384235581390893', '6006627476560013656', '-5872443091340192918', '2339809570377332086']
